#### Import Libraries

In [ ]:
# Import essential libraries for modeling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Set plotting style
plt.style.use('default')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

####  Load and Examine Cleaned Dataset

In [ ]:
# Load the cleaned dataset
df = pd.read_csv('../data/train_cleaned.csv', keep_default_na=False, na_values=[])

print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Missing values: {df.isnull().sum().sum()}")

# Display basic info
print(f"\nData Types:")
print(f"Numerical features: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"Categorical features: {len(df.select_dtypes(exclude=[np.number]).columns)}")

# Show first few rows
print(f"\nFirst 3 rows:")
df.head(3)

#### Correlation Analysis with Target Variable

In [ ]:
# Select only numerical features for correlation analysis
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Found {len(numerical_features)} numerical features")

# Calculate correlation with target variable (SalePrice) - numerical features only
correlations = df[numerical_features].corr()['SalePrice'].sort_values(ascending=False)

print("\nTOP 15 FEATURES MOST CORRELATED WITH SALEPRICE:")
print("=" * 55)
top_15 = correlations.head(16)[1:]  # Exclude SalePrice itself
for feature, corr in top_15.items():
    print(f"{feature:<25} {corr:>8.3f}")

print(f"\nTOP 10 NEGATIVE CORRELATIONS:")
print("=" * 55)
bottom_10 = correlations.tail(10)
for feature, corr in bottom_10.items():
    print(f"{feature:<25} {corr:>8.3f}")

print(f"\nStrong correlations (|r| > 0.5): {len(correlations[correlations.abs() > 0.5]) - 1}")
print(f"Moderate correlations (0.3 < |r| < 0.5): {len(correlations[(correlations.abs() > 0.3) & (correlations.abs() <= 0.5)])}")

TOP PERFORMERS:

    TotalSF (0.782) - engineered total square footage feature is #2

    TotalBath (0.613) - bathroom count feature made top 10

    ExteriorQualityAvg (0.590) - quality average is working great


KEY INSIGHTS:
    
    20 strong correlations (|r| > 0.5) - Excellent feature set

    Quality features dominate - OverallQual, ExterQual_Ordinal, KitchenQual_Ordinal

    Size matters most - TotalSF, GrLivArea, GarageArea all high

    Age is negative - Older houses worth less (HouseAge: -0.523)

#### Analyze Your Engineered Features Performance

In [ ]:
# Identify and analyze engineered features
engineered_features = ['TotalSF', 'TotalBath', 'TotalPorchSF', 'HouseAge', 'YearsSinceRemod', 
                      'GarageAge', 'WasRemodeled', 'LivingAreaRatio', 'BasementRatio',
                      'ExteriorQualityAvg', 'BasementQualityAvg', 'HasPool', 'HasGarage', 
                      'HasBasement', 'HasFireplace', 'Has2ndFloor', 'HasMasVnr', 
                      'HasWoodDeck', 'HasOpenPorch']

# Get correlations for engineered features
eng_correlations = correlations[engineered_features].sort_values(ascending=False, key=abs)

print("ENGINEERED FEATURES PERFORMANCE:")
print("=" * 55)
for feature, corr in eng_correlations.items():
    status = "🔥" if abs(corr) > 0.5 else "✅" if abs(corr) > 0.3 else "📊"
    print(f"{status} {feature:<20} {corr:>8.3f}")

print(f"\nEngineered features with strong correlation: {len(eng_correlations[eng_correlations.abs() > 0.5])}")
print(f"Engineered features with moderate correlation: {len(eng_correlations[(eng_correlations.abs() > 0.3) & (eng_correlations.abs() <= 0.5)])}")


STAR PERFORMERS:

    TotalSF (0.782) - best engineered feature!

    TotalBath (0.613) - Bathroom count is highly predictive

    Quality averages - Both exterior and basement quality averages work great

    Age features - Negative correlations make perfect sense (newer = more valuable)

#### Visualize Top Correlations

In [ ]:
# Create correlation visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Top positive correlations
top_positive = correlations.head(16)[1:]  # Exclude SalePrice itself
top_positive.plot(kind='barh', ax=ax1, color='skyblue')
ax1.set_title('Top 15 Positive Correlations with SalePrice', fontsize=14, fontweight='bold')
ax1.set_xlabel('Correlation Coefficient')
ax1.grid(axis='x', alpha=0.3)

# Top negative correlations
top_negative = correlations.tail(10)
top_negative.plot(kind='barh', ax=ax2, color='lightcoral')
ax2.set_title('Top 10 Negative Correlations with SalePrice', fontsize=14, fontweight='bold')
ax2.set_xlabel('Correlation Coefficient')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"CORRELATION ANALYSIS SUMMARY:")
print(f"   • Total numerical features analyzed: {len(correlations)-1}")
print(f"   • Strong positive correlations (r > 0.5): {len(correlations[correlations > 0.5])-1}")
print(f"   • Strong negative correlations (r < -0.5): {len(correlations[correlations < -0.5])}")
print(f"   • Your engineered features in top 15: {len([f for f in top_positive.index if f in engineered_features])}")


KEY INSIGHTS FROM THE CHART:
    
    4 of the engineered features made it into the top 15 (TotalSF, TotalBath, ExteriorQualityAvg, BasementQualityAvg)
    
    TotalSF is #2 - best engineered feature, beating many original features
    
    Age features dominate negatives - HouseAge and YearsSinceRemod are the strongest negative predictors

#### Correlation Heatmap for Top Features

In [ ]:
# Create correlation heatmap for top features (excluding target)
top_features = correlations.abs().sort_values(ascending=False).head(21).index.tolist()
top_features.remove('SalePrice')  # Remove target variable

print(f"Analyzing multicollinearity among top {len(top_features)} features (excluding SalePrice)")

# Calculate correlation matrix for top features
correlation_matrix = df[top_features].corr()

# Create heatmap
plt.figure(figsize=(16, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f')
plt.title('Feature-to-Feature Correlation Heatmap - Top 20 Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
# plt.savefig("correlation_heatmap.png", dpi=300, bbox_inches='tight')

# Identify highly correlated feature pairs (potential multicollinearity)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:  # High correlation threshold
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], corr_val))

print(f"\nHIGH CORRELATION PAIRS (|r| > 0.8) - Potential Multicollinearity:")
for feat1, feat2, corr in high_corr_pairs:
    print(f"   {feat1} ↔ {feat2}: {corr:.3f}")


Strong correlations you found

    TotalSF ↔ GrLivArea (0.874) → Not surprising: TotalSF includes both basement + above-ground area, while GrLivArea is above-ground only.

    TotalSF ↔ TotalBsmtSF (0.827) and TotalSF ↔ 1stFlrSF (0.800) → Same story: TotalSF aggregates other features.

    GrLivArea ↔ TotRmsAbvGrd (0.825) → Makes sense: more above-ground area usually means more rooms.

    ExterQual_Ordinal ↔ ExteriorQualityAvg (0.855) and BsmtQual_Ordinal ↔ BasementQualityAvg (0.840) → These are engineered averages that directly include those ordinals → they should be correlated.
    
    GarageCars ↔ GarageArea (0.882) → Very strong: more garage spaces = more area. Redundant predictors.
    
    TotalBsmtSF ↔ 1stFlrSF (0.820) → Larger houses often have both big basements and big first floors.
    
    HouseAge ↔ YearBuilt (-1.000) → Perfect negative correlation because one is derived from the other.
    
    YearRemodAdd ↔ YearsSinceRemod (-1.000) → Same: perfect inverse because one is the complement of the other.

What this means for your model

    Multicollinearity alert : Features like TotalSF, GrLivArea, 1stFlrSF, and TotalBsmtSF are almost telling the same story. Linear models (like regression, Lasso, Ridge) could be unstable if we include all of them.
    
    Engineered features vs originals: For example, ExterQual_Ordinal and ExteriorQualityAvg → we probably only need one. Same for HouseAge vs YearBuilt.
    
    Tree-based models (Random Forest, XGBoost, LightGBM) are less sensitive, but even then, redundant features can dilute importance.

In practice:

    For linear models: drop some of the redundant ones or use regularization (Lasso).
    
    For tree models: we can keep them, but still worth pruning for efficiency.
    
    For feature selection: check Variance Inflation Factor (VIF) to confirm multicollinearity.

#### VIF Analysis for Multicollinearity Detection

In [ ]:
# VIF Analysis - Variance Inflation Factor
def calculate_vif(df_features):
    """Calculate VIF for each feature"""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df_features.columns
    vif_data["VIF"] = [variance_inflation_factor(df_features.values, i) 
                       for i in range(len(df_features.columns))]
    return vif_data.sort_values('VIF', ascending=False)

# Select top correlated features for VIF analysis (excluding SalePrice)
top_features_for_vif = correlations.abs().sort_values(ascending=False).head(21).index.tolist()
top_features_for_vif.remove('SalePrice')

print(f"Calculating VIF for top {len(top_features_for_vif)} features...")

# Calculate VIF
vif_results = calculate_vif(df[top_features_for_vif])

print("\nVIF ANALYSIS RESULTS:")
print("=" * 40)
print("VIF > 10: High multicollinearity")
print("VIF > 5: Moderate multicollinearity") 
print("VIF < 5: Acceptable")
print("=" * 40)

for _, row in vif_results.iterrows():
    if row['VIF'] > 10:
        status = "🚨 HIGH"
    elif row['VIF'] > 5:
        status = "⚠️  MOD"
    else:
        status = "✅ OK"
    print(f"{status} {row['Feature']:<25} VIF: {row['VIF']:>8.2f}")

# Count issues
high_vif = len(vif_results[vif_results['VIF'] > 10])
mod_vif = len(vif_results[(vif_results['VIF'] > 5) & (vif_results['VIF'] <= 10)])

print(f"\n📈 SUMMARY:")
print(f"   High multicollinearity (VIF > 10): {high_vif} features")
print(f"   Moderate multicollinearity (VIF 5-10): {mod_vif} features")

VIF Analysis Insights:

    CRITICAL ISSUES (VIF = inf):
        Perfect multicollinearity detected! VIF = infinity means perfect linear relationships
        YearsSinceRemod ↔ YearRemodAdd: Perfectly correlated (one is derived from the other)
        HouseAge ↔ YearBuilt: Perfectly correlated (HouseAge = 2023 - YearBuilt)

    SEVERE ISSUES (VIF > 100):
        TotalSF (312.88): Engineered feature includes components that are also in the model
        GrLivArea (126.61): Part of TotalSF calculation
        TotalBsmtSF (93.90): Also part of TotalSF calculation
        
    MODERATE ISSUES:
        Garage features, exterior quality - manageable levels

#### Remove Redundant Features

In [ ]:
# Remove features with perfect multicollinearity (VIF = inf)
print("🔧 REMOVING REDUNDANT FEATURES:")

features_to_remove = []

# Remove original features when we have engineered versions
redundant_pairs = [
    ('YearBuilt', 'HouseAge', 'Keep HouseAge (more intuitive)'),
    ('YearRemodAdd', 'YearsSinceRemod', 'Keep YearsSinceRemod (more intuitive)'),
    ('GrLivArea', 'TotalSF', 'Keep TotalSF (our best engineered feature)'),
    ('TotalBsmtSF', 'TotalSF', 'Keep TotalSF (comprehensive measure)')
]

for original, engineered, reason in redundant_pairs:
    if original in df.columns:
        features_to_remove.append(original)
        print(f"Remove {original} → Keep {engineered} ({reason})")

print(f"\nFeatures to remove: {len(features_to_remove)}")
print(f"Features: {features_to_remove}")

# Create cleaned feature set
features_cleaned = [col for col in top_features_for_vif if col not in features_to_remove]
print(f"\n✅ Cleaned feature set: {len(features_cleaned)} features")

#### Verify VIF Improvement with Cleaned Features

In [ ]:
# Recalculate VIF with cleaned feature set
print(f"Recalculating VIF with {len(features_cleaned)} cleaned features...")
print(f"Removed features: {features_to_remove}")

# Calculate VIF for cleaned features
vif_cleaned = calculate_vif(df[features_cleaned])

print(f"\nVIF RESULTS AFTER CLEANING:")
print("=" * 40)

for _, row in vif_cleaned.iterrows():
    if row['VIF'] > 10:
        status = "🚨 HIGH"
    elif row['VIF'] > 5:
        status = "⚠️  MOD"
    else:
        status = "✅ OK"
    print(f"{status} {row['Feature']:<25} VIF: {row['VIF']:>8.2f}")

# Compare before/after
high_vif_before = 7  # From previous analysis
high_vif_after = len(vif_cleaned[vif_cleaned['VIF'] > 10])

print(f"\nIMPROVEMENT:")
print(f"   High VIF features before: {high_vif_before}")
print(f"   High VIF features after: {high_vif_after}")
print(f"   Improvement: {high_vif_before - high_vif_after} fewer problematic features")


Key Issues Revealed:
        
        ExteriorQualityAvg ↔ ExterQual_Ordinal (269.67 vs 204.16): average includes the ordinal
        
        BasementQualityAvg ↔ BsmtQual_Ordinal (66.81 vs 82.30): Same issue
        
        Area features still correlated: TotalSF, 1stFlrSF, TotRmsAbvGrd

We have groups of highly correlated features that essentially measure the same thing. Keeping all of them creates multicollinearity, but we need to keep the best representative from each group.

The Strategy:

    1. Group Similar Features:
    2. Selection Criteria:
        Highest correlation with target (SalePrice)
        Less multicollinear (simpler features often better)
        More interpretable (easier to understand)
        Engineered features we created (if they're better)

In [ ]:
# Strategic feature selection - keep the best from each correlated group
print("STRATEGIC FEATURE SELECTION:")

# Define feature groups and select best from each
feature_selection_strategy = {
    'Size Features': {
        'candidates': ['TotalSF', '1stFlrSF', 'TotRmsAbvGrd'],
        'keep': 'TotalSF',  # Highest correlation with target
        'reason': 'Best engineered feature (r=0.782)'
    },
    'Exterior Quality': {
        'candidates': ['ExterQual_Ordinal', 'ExteriorQualityAvg'],
        'keep': 'ExterQual_Ordinal',  # Original, less complex
        'reason': 'Simpler, less multicollinear'
    },
    'Basement Quality': {
        'candidates': ['BsmtQual_Ordinal', 'BasementQualityAvg'],
        'keep': 'BsmtQual_Ordinal',  # Original, less complex
        'reason': 'Simpler, less multicollinear'
    },
    'Garage Features': {
        'candidates': ['GarageCars', 'GarageArea'],
        'keep': 'GarageCars',  # More interpretable
        'reason': 'More interpretable than area'
    },
    'Bathroom Features': {
        'candidates': ['FullBath', 'TotalBath'],
        'keep': 'TotalBath',  # Your engineered feature
        'reason': 'Comprehensive bathroom count'
    }
}

# Apply selection strategy
final_features = []
removed_features = []

for group_name, strategy in feature_selection_strategy.items():
    keep_feature = strategy['keep']
    candidates = strategy['candidates']
    
    final_features.append(keep_feature)
    removed_features.extend([f for f in candidates if f != keep_feature])
    
    print(f"\n{group_name}:")
    print(f"Keep: {keep_feature} - {strategy['reason']}")
    print(f"Remove: {[f for f in candidates if f != keep_feature]}")

# Add remaining features that weren't in groups
other_features = [f for f in features_cleaned if f not in sum([s['candidates'] for s in feature_selection_strategy.values()], [])]
final_features.extend(other_features)

print(f"\nFINAL FEATURE SET:")
print(f"   Selected features: {len(final_features)}")
print(f"   Features: {final_features}")

#### Final VIF Verification

In [ ]:
# Final VIF check on strategically selected features
print(f"Final VIF check on {len(final_features)} strategically selected features...")

# Calculate VIF for final feature set
vif_final = calculate_vif(df[final_features])

print(f"\nFINAL VIF RESULTS:")
print("=" * 40)

acceptable_count = 0
moderate_count = 0
high_count = 0

for _, row in vif_final.iterrows():
    if row['VIF'] > 10:
        status = "🚨 HIGH"
        high_count += 1
    elif row['VIF'] > 5:
        status = "⚠️  MOD"
        moderate_count += 1
    else:
        status = "✅ OK"
        acceptable_count += 1
    print(f"{status} {row['Feature']:<20} VIF: {row['VIF']:>8.2f}")

print(f"\nFINAL MULTICOLLINEARITY SUMMARY:")
print(f"Acceptable (VIF < 5): {acceptable_count} features")
print(f"Moderate (VIF 5-10): {moderate_count} features") 
print(f"High (VIF > 10): {high_count} features")

# Compare with original
print(f"\nIMPROVEMENT SUMMARY:")
print(f"   Original features: 20 → Final features: {len(final_features)}")
print(f"   High VIF features: 13 → {high_count}")
print(f"   Feature reduction: {20 - len(final_features)} features removed")

# Show correlation with target for final features
print(f"\nFINAL FEATURES CORRELATION WITH SALEPRICE:")
final_correlations = correlations[final_features].sort_values(ascending=False, key=abs)
for feature, corr in final_correlations.items():
    strength = "🔥" if abs(corr) > 0.6 else "✅" if abs(corr) > 0.4 else "📊"
    print(f"{strength} {feature:<20} r = {corr:>7.3f}")


#### Phase 2B Summary and Final Feature Set

In [ ]:
# Phase 2B Complete - Prepare final feature set for modeling
print("=" * 60)
print("PHASE 2B COMPLETE: FEATURE SELECTION SUMMARY")
print("=" * 60)

# Create final modeling dataset
X_features = final_features
y_target = 'SalePrice'

print(f"\nFINAL MODELING DATASET:")
print(f"   Features (X): {len(X_features)} carefully selected features")
print(f"   Target (y): {y_target}")
print(f"   Dataset shape: {df.shape}")

print(f"\nTOP PREDICTIVE FEATURES:")
for i, (feature, corr) in enumerate(final_correlations.head(5).items(), 1):
    print(f"   {i}. {feature}: r = {corr:.3f}")

print(f"\nMULTICOLLINEARITY STATUS:")
print(f"   • Removed 10 redundant features")
print(f"   • Reduced high VIF from 13 to 7 features")
print(f"   • Kept all features with strong target correlation")
print(f"   • Ready for robust modeling")

print(f"\nREADY FOR PHASE 3: MODEL BUILDING")
print(f"   • Linear models (Ridge, Lasso) will handle remaining multicollinearity")
print(f"   • Tree-based models (Random Forest) are robust to multicollinearity")
print(f"   • Feature set optimized for predictive power")

# Save final feature list for modeling
final_feature_set = {
    'features': X_features,
    'target': y_target,
    'correlations': final_correlations.to_dict()
}

print(f"\nFeature set ready for modeling phase!")
